In [1]:
from reinforcement_training import *
from notebook_game_helper import *

Using Theano backend.


In [2]:
model0, log = main(training_game_size = 8, training_episodes = 250, steps = 50)

Episode 249, Wins: 157, Replay Loss: 9.4378: 100%|██████████| 250/250 [03:24<00:00,  1.28it/s] 


In [ ]:
val_game = NeuralNaviGame(8, 8, model = model0)

In [ ]:
val_game.setup()

In [ ]:
animate_game(val_game, n = 10)

In [ ]:
val_game.shift_goal()

In [ ]:
f, axarr = pl.subplots(2, 2)
# f.title("RL Loss, 100 eps w/ 50 steps, Look: 20")
x = np.linspace(0, len(log[0]), len(log[0]))
axarr[0, 0].plot(x, np.log(log[0]))
axarr[0, 0].set_title('Log Total Loss')
axarr[0, 1].plot(x, log[0])
axarr[0, 1].set_title('Total Loss')
axarr[1, 0].plot(x, np.log(log[1]))
axarr[1, 0].set_title('Log Replay Loss')
axarr[1, 1].plot(x, log[1])
axarr[1, 1].set_title('Replay Loss')
f.subplots_adjust(hspace=0.5)

pl.plot()
pl.savefig("notes/rl_curve_x.png")
pl.show()

In [ ]:
len(log[0]) == len(log[1])